<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/srihari-dev/notebooks/model_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install transformers cloud-tpu-client torch
# !wget 'https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar'
# !tar -xvf 'scibert_scivocab_uncased.tar'

In [2]:
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Dense, Activation, Concatenate, Dropout
from transformers import TFBertModel
from time import time
print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0-dev20191226


In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver('srihari-1-tpu')  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['192.168.19.2:8470']


INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: srihari-1-tpu


INFO:tensorflow:Initializing the TPU system: srihari-1-tpu


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [4]:
batch_size = 32 * strategy.num_replicas_in_sync
embedding_dim = 512
autotune = tf.data.experimental.AUTOTUNE

train_steps = 1262996 * 0.8 // batch_size
val_steps = train_steps // 10
epochs = 100
print('Batch Size:', batch_size)

Batch Size: 256


In [5]:
base_dir = 'gs://tfworld'
model_dir = os.path.join(base_dir, 'model_files_2')
tensorboard_dir = os.path.join(model_dir, 'logs_'+str(time()))
tfrecords_pattern_train = os.path.join(base_dir, 'tfrecords', 'train*')
tfrecords_pattern_val = os.path.join(base_dir, 'tfrecords', 'eval*')

print('Logging in: ', tensorboard_dir)

Logging in:  gs://tfworld/model_files_2/logs_1577426847.0237095


In [6]:
features = {
    'title':tf.io.FixedLenFeature([512], dtype=tf.int64),
    'citation':tf.io.FixedLenFeature([512], dtype=tf.float32),
    }

def parse_example(example_proto):
    parsed_example = tf.io.parse_single_example(example_proto, features)
    title = parsed_example['title']
    citation = parsed_example['citation']
    
    title = tf.cast(title, dtype=tf.int32)
    citation = tf.cast(citation, dtype=tf.float32)
    return (title, citation), tf.constant([1.0], dtype=tf.float32)

In [7]:
with strategy.scope():
    train_files = tf.data.Dataset.list_files(tfrecords_pattern_train)
    train_dataset = train_files.interleave(tf.data.TFRecordDataset,
                                           cycle_length=32,
                                           block_length=4,
                                           num_parallel_calls=autotune)
    train_dataset = train_dataset.map(parse_example, num_parallel_calls=autotune)
    train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
    train_dataset = train_dataset.repeat()
    train_dataset = train_dataset.prefetch(autotune)

    val_files = tf.data.Dataset.list_files(tfrecords_pattern_val)
    val_dataset = val_files.interleave(tf.data.TFRecordDataset,
                                       cycle_length=32,
                                       block_length=4,
                                       num_parallel_calls=autotune)
    val_dataset = val_dataset.map(parse_example, num_parallel_calls=autotune)
    val_dataset = val_dataset.batch(batch_size, drop_remainder=True)
    val_dataset = val_dataset.repeat()
    val_dataset = val_dataset.prefetch(autotune)

tf.data.experimental.get_structure(train_dataset), tf.data.experimental.get_structure(val_dataset)

(((TensorSpec(shape=(256, 512), dtype=tf.int32, name=None),
   TensorSpec(shape=(256, 512), dtype=tf.float32, name=None)),
  TensorSpec(shape=(256, 1), dtype=tf.float32, name=None)),
 ((TensorSpec(shape=(256, 512), dtype=tf.int32, name=None),
   TensorSpec(shape=(256, 512), dtype=tf.float32, name=None)),
  TensorSpec(shape=(256, 1), dtype=tf.float32, name=None)))

In [8]:
@tf.function
def loss_fn(_, probs):
    '''
        1. Every sample is its own positive, and  the rest of the
            elements in the batch are its negative.
        2. Each TPU core gets 1/8 * global_batch_size elements, hence
            compute shape dynamically.
        3. Dataset produces dummy labels to make sure the loss_fn matches
            the loss signature of keras, actual labels are computed inside this
            function.
        4. `probs` lie in [0, 1] and are to be treated as probabilities.
    '''
    bs = tf.shape(probs)[0] 
    labels = tf.eye(bs, bs)
    return tf.losses.categorical_crossentropy(labels, probs, from_logits=False)
    
def create_model(drop_out):
    textIds = tf.keras.Input(shape=(512,), dtype=tf.int32)    # from bert tokenizer
    citation = tf.keras.Input(shape=(512,))                   # normalized word2vec outputs
    
    bert_model = TFBertModel.from_pretrained('scibert_scivocab_uncased', from_pt=True)
    
    textOut = bert_model(textIds)
    textOutMean = tf.reduce_mean(textOut[0], axis=1)
    textOutSim = Dense(units=embedding_dim, activation='tanh', name='DenseTitle')(textOutMean)
    textOutSim = Dropout(drop_out)(textOutSim)
    
    citationSim = Dense(units=embedding_dim, activation='tanh', name='DenseCitation')(citation)
    citationSim = Dropout(drop_out)(citationSim)

    # Get dot product of each of title x citation combinations
    dotProduct = tf.reduce_sum(tf.multiply(textOutSim[:, None, :], citationSim), axis=-1)
    
    # Softmax to make sure each row has sum == 1.0
    probs = tf.nn.softmax(dotProduct, axis=-1)

    model = tf.keras.Model(inputs=[textIds, citation], outputs=[probs])
    return model

In [9]:
with strategy.scope():
    model = create_model(drop_out=0.20)
    model.compile(loss=loss_fn,
                  optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5))

In [10]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=tensorboard_dir, update_freq='epoch'), 
            tf.keras.callbacks.ModelCheckpoint(filepath=model_dir + '/epoch_{epoch:02d}_{loss:.2f}',
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_freq='epoch')
                                               ]

In [ ]:
model.fit(train_dataset,
          epochs=epochs,
          steps_per_epoch=train_steps,
          validation_data=val_dataset,
          validation_steps=val_steps,
          validation_freq=1,
          callbacks=callbacks)

Train for 3946.0 steps, validate for 394.0 steps


Epoch 1/100


   1/3946 [..............................] - ETA: 74:42:48 - loss: 3.7574WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.095701). Check your callbacks.


3945/3946 [============================>.] - ETA: 0s - loss: 3.0831
Epoch 00001: saving model to gs://tfworld/model_files_2/epoch_01_3.08
3946/3946 [==============================] - 2403s 609ms/step - loss: 3.0830 - val_loss: 2.7980
Epoch 2/100
3945/3946 [============================>.] - ETA: 0s - loss: 2.7259
Epoch 00002: saving model to gs://tfworld/model_files_2/epoch_02_2.73
3946/3946 [==============================] - 2362s 599ms/step - loss: 2.7259 - val_loss: 2.5801
Epoch 3/100
3945/3946 [============================>.] - ETA: 0s - loss: 2.4969
Epoch 00003: saving model to gs://tfworld/model_files_2/epoch_03_2.50
3946/3946 [==============================] - 2363s 599ms/step - loss: 2.4969 - val_loss: 2.4444
Epoch 4/100
3945/3946 [============================>.] - ETA: 0s - loss: 2.3048
Epoch 00004: saving model to gs://tfworld/model_files_2/epoch_04_2.30
3946/3946 [==============================] - 2359s 598ms/step - loss: 2.3047 - val_loss: 2.3618
Epoch 5/100
3945/3946 [=====